In [3]:

# only run this once if required and margin list has changes from website!!! 
# just run everything else below this cell on regular basis

# import margin from westpac website
import tabula
import pandas as pd
myfile = 'https://www.westpac.com.au/content/dam/public/wbc/documents/pdf/pb/1999030/WOIL_Acceptable_Securities_List.pdf'

df2 = tabula.read_pdf(myfile,pages='all')
frames = [df2[0],df2[1],df2[2],df2[3]]
df2 = pd.concat(frames)
df2.drop(['Unnamed: 0','Unnamed: 1','APIR Code','Fund Name','APIR Code Fund Name'],axis=1) 
split1 = df2[['ASX Code','Security Name','LVR']]
split2 = df2[['ASX Code.1','Security Name.1','LVR.1']]
split2.columns=['ASX Code','Security Name','LVR']
df2 = pd.concat([split1,split2])
df2 = df2.dropna()
df2 = df2.drop_duplicates(subset=['ASX Code'])
#df2.loc[:,'ASX Code']=df2['ASX Code'] + '.AX'


a = pd.read_csv('margin.csv')
df = pd.merge(a,df2,on='ASX Code',how='left')
df = df.drop(['LVR_x','Security Name_x'],axis=1)
df = df.rename(columns={'Security Name_y':'Security Name','LVR_y':'LVR'})
df = df[['ASX Code','Security Name','LVR','Industry','Valuation']]
#df = df.style.set_properties(**{'text-align': 'left'})

df.to_csv('marg.csv')

In [121]:
import pandas as pd
import datetime
import math
import numpy as np
#import pandas_datareader.data as web
from pandas import Series, DataFrame
from sklearn import preprocessing
import matplotlib.pyplot as plt
from datetime import date, timedelta
import yfinance as yf

pd.set_option('mode.chained_assignment',None)    # turn off chain error warning since I use loc function properly
pd.options.display.float_format = '{:,.2f}'.format


# myfunction to use MACD to give signal


# 1. look at mac
# 2. look at overall market 

def MAC(MACDiff,MACD,Trigger,Movement):
    if MACDiff >= 0.01:
        if MACD < 0 and MACD < Trigger:
            action = 'Buy - accumulate'
        elif MACD < 0 and MACD > Trigger and Movement > 0:
            action = 'Buy - positive trigger'
        elif MACD < 0 and MACD > Trigger and Movement < 0:
            action = 'Buy - weak trigger'
        elif MACD > 0 and MACD > Trigger:
            action = 'Buy - positive signal'
        else:
            action = 'MACD increases but no comment'
    elif MACDiff > -0.01 and MACDiff < 0.01:
        if Movement > 0:
            action = 'MACD flat with buy activity'
        elif Movement < 0:
            action = 'MACD flat with sell activity'
        else:
            action = 'Sideway'
    else:    # MACDIFF IS NEGATIVE
        if MACD > 0:
            action = 'Buy - but momentum declines'    
        elif MACD < 0 and MACD < Trigger:
            action = 'Sell'
        elif MACD < 0 and Movement > 0:                 
            action = 'Sell - a bit of buy support'       #technical bad but ppl still buy? must be good value
        else:
            action = 'MACD declines'
    return action



# my purchase price table
myInput = pd.DataFrame({'Code':['AGL.AX','WPL.AX','WBC.AX'],
                        'PurchasePrice': [13.535,19.98,21.35],
                        'Unit': [4447,1275,163],
                        'InPortfolio':['y','y','y']
                       })

# pick up data from yahoo finance
start = datetime.datetime(2020, 7, 1)
end = datetime.date.today() + datetime.timedelta(days=1)
#end = datetime.datetime(2020,10,14)


# new code to pick up from csv instead with more details
mycsv = 'https://raw.githubusercontent.com/RonnyCh/mydsbook/master/margin.csv'
mystock = pd.read_csv(mycsv)
mystock = mystock['ASX Code']
#mystock = mystock.replace('^AORD.AX','^AORD')    # all ord does not come with .ax

# convert to string to make it better with dowloanding tracker
mystring = ''
mylist = []
for i in mystock:
    if i in ['^AORD','^DJI','^FTSE','CL=F','^AXJO','^IXIC','^HIS','^N225']:     # indexes no need to add .AX
        mystring = mystring + ' ' + i
        mylist.append(i)
    else:
        mystring = mystring + ' ' + i + '.AX'
        mylist.append(i+'.AX')

# download data 
data = yf.download(mystring, start=start, end=end, group_by="ticker")


# create columns for dataframe and the dataframe itself
mycol = ['Code','Close','Movement','Volume','ShortAvg','LongAvg','MACD','Trigger','Max','Min','2DAvg','VolDiff','P/Max']
mytbl = pd.DataFrame(columns=mycol)


# looping through the list to modify table
for i in mylist:
    #df = web.DataReader(i, 'yahoo', start, end)[['Close','Volume']]    # old code using datareader (not working well)
    #df = yf.download(i, start=start, end=end)[['Close','Volume']]      # old Yfinance code
    
    df = data[i][['Close','Volume']]     # new one using Yfinance
    df['Code'] = i
   #df.loc[:,'ShortAvg'] = df.Close.rolling(window=7).mean()
    df.loc[:,'ShortAvg'] = df.Close.ewm(span=5).mean()        
    df.loc[:,'LongAvg'] = df.Close.ewm(span=14).mean()
    df['MACD'] = df['ShortAvg'] - df['LongAvg']
    df['Trigger'] = df.MACD.rolling(window=4).mean()
    df['Movement'] = df['Close'].diff(1)
    df['2DAvg'] = df.Movement.rolling(window=2).sum()
    df['Max'] = df['Close'].max()
    df['Min'] = df['Close'].min()
    df['VolDiff'] = df['Volume'].diff(1)
    df['P/Max'] = df['Close']/df['Max']
    mytbl = mytbl.append(df)

mytbl = mytbl.reset_index()
mytbl = mytbl.rename(columns={'index':'Date'})


############## SQL TO MERGE WITH MY INPUT ##############
code = pd.merge(mytbl,myInput,on='Code',how='left')
code = code.rename(columns={'index':'Date'})


#remove null values
code['Unit'] = code['Unit'].fillna(0)
code['PurchasePrice'] = code['PurchasePrice'].fillna(0)
code['InPortfolio'] = code['InPortfolio'].fillna('n')

code['Profit'] = (code['Close'] - code['PurchasePrice']) * code['Unit']
#code['AvgPctg'] = (code['Wkly']-code['TwoWks'])/code['TwoWks'] * 100

#code['Diff'] = code['Wkly'] - code['TwoWks']
#code['DdownCnt']=code['Ddown'].rolling(window=14,min_periods=1).count()




# create another table to give signal
#mysignal = code
#mysignal['MACD_Diff'] = mysignal['MACD'].diff(1)
#mysignal['Advice']=mysignal.apply(lambda x:MAC(x['MACD_Diff'],x['MACD'],x['Trigger'],x['Movement']),axis=1)




code['MACD_Diff'] = code['MACD'].diff(1)
code['Advice']=code.apply(lambda x:MAC(x['MACD_Diff'],x['MACD'],x['Trigger'],x['2DAvg']),axis=1)


code.to_csv('YahooFinance.csv')


#code.head()






#code.to_csv('mysignal.csv')





[*********************100%***********************]  399 of 399 completed


In [122]:
import datetime as dt
start = datetime.date.today() + datetime.timedelta(days=-7)
start = start.strftime("%m/%d/%Y")
end = dt.datetime.today().strftime("%m/%d/%Y")
mydate = pd.date_range(start,end)



mycol = ['Date','Code','Close','MACD','Trigger','MACD_Diff','Movement','2DAvg','Min','Max','P/Max','Advice']
rec = code[mycol]

print ('******Your Portfolio Index Performance************')

rec[(rec['Code'].isin(['^AORD','^DJI','WPL.AX','AGL.AX','IAG.AX','TWE.AX','LNK.AX','CSL.AX','ALL.AX','RMD.AX'])) & (rec['Date']>start) & (rec['Date']<=end)].sort_values(by=['Code','Date'],ascending=False)

******Your Portfolio Index Performance************


,Date,Code,Close,MACD,Trigger,MACD_Diff,Movement,2DAvg,Min,Max,P/Max,Advice
607,2021-01-15,^DJI,nan,232.31,254.61,0.00,nan,nan,"25,706.09","31,097.97",nan,Sideway
606,2021-01-14,^DJI,"30,991.52",232.31,269.25,-35.83,-68.95,-77.17,"25,706.09","31,097.97",1.00,Buy - but momentum declines
605,2021-01-13,^DJI,"31,060.47",268.14,285.53,-17.56,-8.22,51.78,"25,706.09","31,097.97",1.00,Buy - but momentum declines
604,2021-01-12,^DJI,"31,068.69",285.69,282.01,-5.18,60.00,-29.28,"25,706.09","31,097.97",1.00,Buy - but momentum declines
603,2021-01-11,^DJI,"31,008.69",290.88,254.76,-6.54,-89.28,-32.44,"25,706.09","31,097.97",1.00,Buy - but momentum declines
1063,2021-01-15,^AORD,"6,986.80",19.43,17.00,1.93,4.10,32.90,"5,973.50","7,024.20",0.99,Buy - positive signal
1062,2021-01-14,^AORD,"6,982.70",17.50,18.53,3.36,28.80,43.60,"5,973.50","7,024.20",0.99,MACD increases but no comment
1061,2021-01-13,^AORD,"6,953.90",14.13,22.07,-2.81,14.80,-5.60,"5,973.50","7,024.20",0.99,Buy - but momentum declines
1060,2021-01-12,^AORD,"6,939.10",16.94,22.84,-8.62,-20.40,-85.10,"5,973.50","7,024.20",0.99,Buy - but momentum declines
1059,2021-01-11,^AORD,"6,959.50",25.56,19.91,-6.07,-64.70,-21.00,"5,973.50","7,024.20",0.99,Buy - but momentum declines


In [123]:
a = rec[(rec['Date']=='2021-01-15') & rec['Advice'].str.contains('trigger') & (rec['P/Max'] < 0.91)]
mystock = pd.read_csv(mycsv)
mystock = mystock.rename(columns={'ASX Code':'Code'})
mystock['Code']=mystock['Code'] + '.AX'
mycol = ['Date','Code','Industry','LVR','Close','MACD','Trigger','2DAvg','Max','P/Max','Advice']
mybuy = pd.merge(a,mystock,on='Code')
mybuy[mycol]
#type(a)

,Date,Code,Industry,LVR,Close,MACD,Trigger,2DAvg,Max,P/Max,Advice
0,2021-01-15,ANN.AX,Health Care Equipment & Services,75%,34.57,-0.16,-0.25,0.49,42.91,0.81,Buy - positive trigger
1,2021-01-15,APX.AX,Software & Services,45%,22.32,-0.91,-0.97,-0.20,43.50,0.51,Buy - weak trigger
2,2021-01-15,AUB.AX,Insurance,60%,15.84,-0.14,-0.19,-0.08,17.72,0.89,Buy - weak trigger
3,2021-01-15,AZJ.AX,Transportation,75%,3.85,-0.04,-0.05,0.05,4.98,0.77,Buy - positive trigger
4,2021-01-15,BVS.AX,Software & Services,40%,2.96,-0.08,-0.10,0.05,4.66,0.64,Buy - positive trigger
5,2021-01-15,CLW.AX,Real Estate,50%,4.50,-0.07,-0.09,0.11,5.23,0.86,Buy - positive trigger
6,2021-01-15,COH.AX,Health Care Equipment & Services,65%,190.80,-1.05,-3.24,10.11,238.52,0.80,Buy - positive trigger
7,2021-01-15,CQR.AX,Real Estate,60%,3.50,-0.07,-0.08,0.06,3.96,0.88,Buy - positive trigger
8,2021-01-15,CUV.AX,"Pharmaceuticals, Biotechnology & Life Sciences",40%,22.31,-0.04,-0.33,1.02,26.10,0.85,Buy - positive trigger
9,2021-01-15,DXS.AX,Real Estate,70%,8.99,-0.17,-0.19,0.10,9.90,0.91,Buy - positive trigger


In [124]:
import numpy as np


#mydate = datetime.date.today().strftime("%Y-%m-%d")
mydate = datetime.date.today() + datetime.timedelta(days=0) #change time delta accordingly, default = 0 or now
# the format of this input : Start, End, Code, Type, Unit, Price
# type --> b = buy , s = sell , w = watchlist
myval = [
    # Bought Date, Sold Date, Code, Type, Unit, Price, Margin
   
    
    ['2020-10-15',mydate,'AGL.AX','Long Term',4447,13.55],
    ['2020-12-15',mydate,'AGL.AX','Short Term',775,12.885],
    ['2020-8-27',mydate,'IAG.AX','Long Term',6060,5.08],
    ['2020-12-04',mydate,'CSL.AX','Long Term',33,294.19], 
    ['2020-12-11',mydate,'CSL.AX','Long Term',68,292.025], 
    ['2020-12-21',mydate,'CSL.AX','Long Term',34,287.511], 
    ['2021-1-12' ,mydate,'CSL.AX','Short Term',54,274.11],     
    ['2020-8-26',mydate,'WPL.AX','Long Term',1275,19.98],
    ['2020-11-25',mydate,'WPL.AX','Long Term',427,23.36],
    ['2020-12-2',mydate,'WPL.AX','Long Term',224,22.24],
    ['2020-12-21',mydate,'TWE.AX','Long Term',607,9.38],
    ['2021-1-4',mydate,'LNK.AX','Long Term',1236,4.78]
    
     ]

# create two tables (manual input vs table for ALLORD)
myinput = pd.DataFrame(myval,columns=['Date','Date_to','Code','Type','Unit','Price'])
myinput['Date']= myinput['Date'].astype('datetime64[ns]')                  # change from objec to the date type
allord = code[code['Code']=='^AORD'][['Date','Close']]



# merge tables to create a consolidated with share vs ordinary

benchmark = pd.merge(myinput,allord,on='Date',how='inner')
#benchmark['']

# loop all the description and create dataframe 
for i in range(len(benchmark['Date'])):
        StartDate = benchmark.loc[i,'Date']
        EndDate = benchmark.loc[i,'Date_to']
        Code = benchmark.loc[i,'Code']
        Unit = benchmark.loc[i,'Unit']
        Price = benchmark.loc[i,'Price']
        Index = benchmark.loc[i,'Close']
        Type = benchmark.loc[i,'Type']
        
        dateseries = pd.date_range(StartDate,EndDate)
        mypurchasedate = np.repeat(StartDate,len(dateseries))
        myunit = np.repeat(Unit,len(dateseries))
        myprice = np.repeat(Price,len(dateseries))
        mycode = Code
        
        if i ==0 : # create the main dataframe the first time
            df1 = pd.DataFrame ({'Date':dateseries, 'Bought_Date':StartDate,'Type':Type,'Code':mycode,'Unit':myunit,'Price':myprice,'Index':Index})
        else:      # create tmp and append to main dataframe
            tmp = pd.DataFrame ({'Date':dateseries, 'Bought_Date':StartDate,'Type':Type,'Code':mycode,'Unit':myunit,'Price':myprice,'Index':Index})
            df1 = df1.append(tmp)
            

            
# create final profit table         

#mydate = datetime.date.today() + datetime.timedelta(days=-1)    #change time delta accordingly, default = 0 or now
mydate = mydate.strftime("%Y-%m-%d")
myprofit =  pd.merge(df1,code,on=['Date','Code'],how='inner')[['Date','Bought_Date','Type','Code','Unit_x','Price','Close','Index']]
myprofit =  pd.merge(myprofit,allord,on='Date',how='inner')
myprofit = myprofit.rename(columns={'Unit_x':'Unit','Close_x':'Close','Close_y':'Index_Close'})
myprofit = myprofit.sort_values(by=['Type'])
myprofit['Index_Unit'] = (myprofit['Unit'] * myprofit['Price'])/myprofit['Index']
myprofit['MyProfit'] = (myprofit['Close'] - myprofit['Price']) * myprofit['Unit']
myprofit['Benchmark'] = (myprofit['Index_Close'] - myprofit['Index']) * myprofit['Index_Unit']
#myprofit['Profit_Impact'] = myprofit['MyProfit'] - myprofit['Benchmark']
myprofit['Profit%'] = (myprofit['Close']-myprofit['Price'])/myprofit['Price']
myprofit['Index%'] = (myprofit['Index_Close']-myprofit['Index'])/myprofit['Index_Close']
myprofit['Diff%'] = myprofit['Profit%'] - myprofit['Index%']  
myprofit['TotCost'] = myprofit['Unit'] * myprofit['Price']  
#myprofit.style.format({'Profit%':"{:.2%}"})




mycolumns = ['Type','Code','Date','Bought_Date','Unit','Price','TotCost','Close','MyProfit','Benchmark','Profit%','Index%','Diff%']
myprofit = myprofit[mycolumns]

myprofit[myprofit['Date'] == mydate]
mydf = myprofit[myprofit['Date']==mydate].sort_values(by=['Type','Code','Bought_Date'])
#myprofit.style.format({'Profit%':"{:.2%}",'Index%':"{:.2%}"})
#myprofit[myprofit['Date'] == mydate]

final = mydf.groupby('Code').sum()[['Unit','TotCost']]
final['AvgCost'] = final['TotCost']/final['Unit']

print ('########## Your Average Costs ############')
print (final)


print ('########### Benchmark against all ordinaries ##############')

mylvr = mystock[['Code','LVR']]

mydf = pd.merge(mydf,mylvr,on='Code',how='left')
#mydf['MarginUsed'] = mydf['LVR'] * mydf['TotCost']
mydf


########## Your Average Costs ############
        Unit   TotCost  AvgCost
Code                           
AGL.AX  5222 70,242.73    13.45
CSL.AX   189 54,143.28   286.47
IAG.AX  6060 30,784.80     5.08
LNK.AX  1236  5,908.08     4.78
TWE.AX   607  5,693.66     9.38
WPL.AX  1926 40,430.98    20.99
########### Benchmark against all ordinaries ##############


,Type,Code,Date,Bought_Date,Unit,Price,TotCost,Close,MyProfit,Benchmark,Profit%,Index%,Diff%,LVR
0,Long Term,AGL.AX,2021-01-15,2020-10-15,4447,13.55,"60,256.85",12.00,"-6,892.85","5,379.17",-0.11,0.08,-0.20,75%
1,Long Term,CSL.AX,2021-01-15,2020-12-04,33,294.19,"9,708.27",267.26,-888.69,171.81,-0.09,0.02,-0.11,75%
2,Long Term,CSL.AX,2021-01-15,2020-12-11,68,292.02,"19,857.70",267.26,"-1,684.02",289.51,-0.08,0.01,-0.10,75%
3,Long Term,CSL.AX,2021-01-15,2020-12-21,34,287.51,"9,775.37",267.26,-688.53,94.36,-0.07,0.01,-0.08,75%
4,Long Term,IAG.AX,2021-01-15,2020-08-27,6060,5.08,"30,784.80",5.05,-181.80,"3,298.68",-0.01,0.10,-0.10,75%
5,Long Term,LNK.AX,2021-01-15,2021-01-04,1236,4.78,"5,908.08",4.66,-148.32,28.12,-0.03,0.00,-0.03,60%
6,Long Term,TWE.AX,2021-01-15,2020-12-21,607,9.38,"5,693.66",9.21,-103.19,54.96,-0.02,0.01,-0.03,65%
7,Long Term,WPL.AX,2021-01-15,2020-08-26,1275,19.98,"25,474.50",26.76,"8,644.50","2,801.81",0.34,0.10,0.24,75%
8,Long Term,WPL.AX,2021-01-15,2020-11-25,427,23.36,"9,974.72",26.76,"1,451.80",142.78,0.15,0.01,0.13,75%
9,Long Term,WPL.AX,2021-01-15,2020-12-02,224,22.24,"4,981.76",26.76,"1,012.48",128.36,0.20,0.03,0.18,75%


In [125]:
print ('### Overall Profit ###')
overall = mydf.groupby('Type').sum()[['Unit','MyProfit']]

print (overall)

mydf.groupby('Code').sum()[['Unit','MyProfit']]

### Overall Profit ###
             Unit  MyProfit
Type                       
Long Term   14411    521.38
Short Term    829 -1,055.77


,Unit,MyProfit
Code,,
AGL.AX,5222,"-7,578.73"
CSL.AX,189,"-3,631.14"
IAG.AX,6060,-181.80
LNK.AX,1236,-148.32
TWE.AX,607,-103.19
WPL.AX,1926,"11,108.78"


In [8]:
pd.options.display.float_format = '{:,.2f}'.format

myprofit = myprofit[myprofit['Date']=='2020-11-06'].sort_values(by=['Type','Code','Bought_Date'])

myprofit
#myprofit.style.format({'Profit%':"{:.2%}",'Index%':"{:.2%}"})

NameError: name 'myprofit' is not defined

In [291]:
 pd.date_range(start, periods=5, freq=pd.offsets.BDay())

DatetimeIndex(['2020-07-01', '2020-07-02', '2020-07-03', '2020-07-06',
               '2020-07-07'],
              dtype='datetime64[ns]', freq='B')

In [317]:
import datetime as dt
s = dt.datetime.today().strftime("%m/%d/%Y")

'11/02/2020'

In [318]:
import datetime as dt
start = datetime.datetime(2020, 7, 1)
end = datetime.date.today() + datetime.timedelta(days=1)

,StartDate,EndDate,Code,Type,Unit,Price
0,2020-10-27,2020-11-02,WPL.AX,B,1120,17.93
1,2020-10-15,2020-11-02,AGL.AX,B,4447,13.54
